In [1]:
import tensorflow as tf
import numpy as np


def round(x):
    return "%.2f" % np.round(x.numpy(), 2)


def SSIM(a, b):
    return round(tf.image.ssim(a, b, 255))


def PSNR(a, b):
    return round(tf.image.psnr(a, b, 255))

In [2]:
# Get validation images

from dlsr import *
from dlsr.data import DIV2K

# from PIL import Image

image_size = 400
loader = DIV2K(type="valid", scale=4)
ds = loader.dataset(
    batch_size=1, random_transform=False, crop_images=True, image_size=image_size
)

hr = ds.map(lambda a, b: b).take(7).cache()
lr = ds.map(lambda a, b: a).take(7).cache()

upscaled = {}

31505881/31505881 [==============================] - 1s 0us/step
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-02-09 16:39:59.542097: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-09 16:39:59.542116: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# # Upscale with my SRResNet x4

model = tf.keras.models.load_model(
    "./results/saved-models/srres4x.h5",
)

upscaled["srres4x"] = upscale(model, lr)

del model
# Upscale with my SRGAN x4

# load json and create model
json_file = open('./results/saved-models/srgan4x/architecture.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("./results/saved-models/srgan4x/generator.h5")

upscaled["srgan4x"] = upscale(model, lr)

del model

# Upscale with my SrResNet x2

model = tf.keras.models.load_model(
    "./results/saved-models/srres2x.h5",
)

temp = upscale(model, lr)
temp = tf.data.Dataset.from_tensors(temp)
upscaled["srres2x"] = upscale(model, temp)
del model

# Upscale with my SRGAN x2

# load json and create model
json_file = open('./results/saved-models/srgan2x/architecture.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("./results/saved-models/srgan2x/generator.h5")

temp = upscale(model, lr)
temp = tf.data.Dataset.from_tensors(temp)
upscaled["srgan2x"] = upscale(model, temp)

del model

2023-02-09 16:40:00.877623: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-09 16:40:01.111818: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-09 16:40:01.472163: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-02-09 16:40:01.533456: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have a

7/7 [==============================] - 2s 134ms/step


2023-02-09 16:40:03.608544: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 2s 136ms/step


2023-02-09 16:40:06.293753: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 487ms/step


2023-02-09 16:40:09.702570: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 389ms/step


In [4]:
# Upscale with Interploation-Based methods

lr_transformed = [np.uint8(x)[0] for x in lr]

new_size = [image_size, image_size]

algs = [
    tf.image.ResizeMethod.NEAREST_NEIGHBOR,
    tf.image.ResizeMethod.BILINEAR,
    tf.image.ResizeMethod.BICUBIC,
    tf.image.ResizeMethod.LANCZOS3,
]

for alg in algs:
    upscaled[alg] = tf.image.resize(
        lr_transformed,
        new_size,
        method=alg,
    )

In [5]:
# save LR images
for i, x in enumerate(lr):
    tf.keras.utils.save_img(path=f"results/comparison4x/lr/image{i}.png", x=x[0], file_format="png")

# save HR images
for i, x in enumerate(hr):
    tf.keras.utils.save_img(path=f"results/comparison4x/hr/image{i}.png", x=x[0], file_format="png")

# save upscaled images
for key in upscaled.keys():
    for i, x in enumerate(upscaled[key]):
        tf.keras.utils.save_img(
            path=f"results/comparison4x/{key}/image{i}.png", x=x, file_format="png"
        )

# calculate SSIM/PSNR
for i, x in enumerate(hr):
    real = np.uint8(x)[0]
    print("-------------------------------------")
    for j, key in enumerate(upscaled.keys()):
        fake = np.uint8(upscaled[key][i])
        print(
            "| {:<4} | {:<10} | {:<4} = {:<6} |".format(
                f"img{i}" if j == 0 else "", f"{key}", "SSIM", SSIM(real, fake)
            )
        )
        print("| {:<4} | {:<10} | {:<4} = {:<6} |".format("", "", "PSNR", PSNR(real, fake)))

2023-02-09 16:40:13.152521: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-02-09 16:40:13.158743: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


-------------------------------------
| img0 | srres4x    | SSIM = 0.96   |
|      |            | PSNR = 37.71  |
|      | srgan4x    | SSIM = 0.93   |
|      |            | PSNR = 33.83  |
|      | srres2x    | SSIM = 0.96   |
|      |            | PSNR = 38.08  |
|      | srgan2x    | SSIM = 0.93   |
|      |            | PSNR = 34.93  |
|      | nearest    | SSIM = 0.82   |
|      |            | PSNR = 30.28  |
|      | bilinear   | SSIM = 0.92   |
|      |            | PSNR = 33.24  |
|      | bicubic    | SSIM = 0.94   |
|      |            | PSNR = 35.28  |
|      | lanczos3   | SSIM = 0.95   |
|      |            | PSNR = 36.17  |
-------------------------------------
| img1 | srres4x    | SSIM = 0.90   |
|      |            | PSNR = 36.22  |
|      | srgan4x    | SSIM = 0.77   |
|      |            | PSNR = 31.81  |
|      | srres2x    | SSIM = 0.91   |
|      |            | PSNR = 36.36  |
|      | srgan2x    | SSIM = 0.72   |
|      |            | PSNR = 30.68  |
|      | nea